In [ ]:
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('dark')
pd.options.display.max_rows = 999

In [ ]:
path = '../../data/'
train_df = pd.read_csv(path + 'train.csv', header=0, names=['classid', 'title', 'description'])
test_df = pd.read_csv(path + 'test.csv', header=0, names=['classid', 'title', 'description'])

In [ ]:
df = pd.concat([train_df, test_df])
df.reset_index(inplace=True, drop=True)

In [ ]:
df.title

In [ ]:
df.title = df.title.str.replace(' #39;', "'")

df.description = df.description.str.replace(' #39;', "'")
df.description = df.description.str.replace(r'\\', ' ') # 2
df.description = df.description.str.replace(r'\\\\', ' ') # 180
df.description = df.description.str.strip(r' \\\n\t\'\"')

df.description = df.description.str.replace(r'\s\s+', ' ', regex=True) 

In [ ]:
 df.title

## Data Cleaning

In [ ]:
df.drop_duplicates(subset=['description'], inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)

df['title_remark'] = ""
df['description_start_0'] = ""
df['description_start_1'] = ""
df['description_start_2'] = ""
df['cleaned_description'] = ""
df['location'] = ""
df.shape

In [ ]:
loc = 170
print(df.iloc[loc].title)
print(df.iloc[loc].description)

In [ ]:
title_remark_re = r'(?:.*)\s\((\w*\.?(?:\w+)+)\)$'
df['title_remark'] = df['title'].str.extract(title_remark_re)


def clean_remark_from_title(row):
    regex = r'(.*)\s\((\w*\.?(?:\w+)+)\)$'
    m = re.search(regex, row)
    if m:
        # m = re.search(regex, row)
        return m.group(1)
    else:
        return row
    
df['title'] = df['title'].apply(clean_remark_from_title)

In [ ]:
df.title

In [ ]:
df.description

In [ ]:
### TEST

def clean_description(row):
    regex = r'''^
                (?:\w*\.?(?:\w+)+)
                (?:\s*-{1,2}\s+)
                (.*)
                |
                (?:(?:\w+\s)+)
                \((?:\w+)\)
                (?:\s*-{1,2}\s+)
                (.*)
            '''
    #regex = '(.*)\s\((\w*\.?(?:\w+)+)\)$'
    
    m = re.search(regex, row, re.M|re.X)
    
    if m:
        print(m.group(1))
        return m.group(1)
    else:
        return row
    
    
df['description'] = df['description'].apply(clean_description)

In [ ]:
df.description

In [ ]:
desc_1_re = r'^(\w+)(?:\s+-{1,2}\s+)(?:.*)'
df['description_start_0'] = df['description'].str.extract(desc_1_re)

"""
desc_2_re = r'^(?:\w+\s)+\((\w+)\)(?:\s+-{1,2}\s+)(?:.*)'
df['description_start_1'] = df['description'].str.extract(desc_2_re)

desc_3_re = r'^(\w*\.?\s?\w+)+(?:\s+-{1,2}\s+)(?:.*)'
df['description_start_2'] = df['description'].str.extract(desc_3_re)
"""

def clean_beginning_from_description(row):
    regex = r'''^
                (?:\w*\.?(?:\w+)+)
                (?:\s*+{1,2}\s+)
                (.*)
                |
                (?:(?:\w+\s)+)
                \((?:\w+)\)
                (?:\s+-{1,2}\s+)
                (.*)
            '''
    m = re.search(regex, row, re.M|re.X)
    if m.group(1):
        #print(re.search(regex, row, re.M|re.X))
        # m = re.search(regex, row, re.M|re.X)
        #print(m.groups())
     
        return m.group(1)
        
    else:
        return row

# df['cleaned_description'] = df['description'].apply(clean_beginning_from_description)
    
"""
# Capture institution (and place) at beginning of description
regex = r'''^
            (\w*\.?(?:\w+)+)
            (?:\s-{0,2}\s)
            (.*)
            |
            ((?:\w+\s)+)
            \((\w+)\)
            (?:\s-{0,2}\s)
            (.*)
        '''

for i in range(len(df)):
    m = re.search(regex, df.iloc[i].description, re.M|re.X)
    if m:
        if m.group(1):
            #print(m.groups())        
            df.iat[i, 4] = m.group(1)
            df.iat[i, 5] = m.group(2)
        elif m.group(3):
            #print(m.groups())
            df.iat[i, 4] = m.group(4)
            df.iat[i, 5] = m.group(5)
            df.iat[i, 6] = m.group(3)
#m = re.search(regex, df.iloc[3].description)
#print(m.groups())
"""

In [ ]:
df

In [ ]:
df['final_description'] = df['cleaned_description'].apply(
    lambda x: x if len(x) > 0 else ""
)

df.loc[df['cleaned_description'] == '', 'final_description'] = df['description']

In [ ]:
df.drop_duplicates(subset=['description'], inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)

In [ ]:
df

## Processing Data

## Splitting Data in Train / Test

In [ ]:
train_data = df.loc[:120000]
test_data = df.loc[120000:]

print(train_data.shape)
print(test_data.shape)
"""
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)
"""